## Работа с запросами к API ВКонтакте средствами библиотеки `requests`

*Алла Тамбовцева, НИУ ВШЭ*

### Введение в API ВКонтакте

Сегодня мы немного поработаем с API. API – программный интерфейс приложения, сокращение от *Application Programming Interface*. Этот интерфейс позволяет выполнять различные операции автоматически, через приложение. Если API нам нужен исключительно как источник данных, можно писать запросы, позволяющие обратиться к хранилищу информации внутри API. Если мы хотим управлять приложением, которое будет выполнять какие-то действия, удаленно, можно написать код, который будет, например, автоматически отвечать на сообщения, когда мы не онлайн, лайкать новый пост друга через 30 секунд после его появления, пересылать на почту фотографии, которые выложили участники диалога и прочее.

Мы будем работать с API социальной сети ВКонтакте. Использовать API для написания и приема сообщений средствами Python мы не будем, а рассмотрим API как источник данных, позволяющий выгрузить данные о пользователи или посты со страницы сообщества.

Для работы нам понадобятся две библиотеки: библиотека `requests` для формирования запросов, и библиотека `pandas` для преобразования словаря, который мы получим на основе JSON-файла, в удобный для работы датафрейм.

In [1]:
import requests
import pandas as pd

Посмотрим на документацию API ВКонтакте и познакомимся с запросами: 

https://vk.com/dev.php?method=api_requests

Если мы посмотрим на пример запроса, увидим, что он представляет собой особого вида ссылку:

`https://api.vk.com/method/METHOD_NAME?PARAMETERS&access_token=ACCESS_TOKEN&v=V`

В этой ссылке мы указываем:

* метод (`METHOD_NAME`): какого вида запрос хотим сделать;
* параметры запроса (`PARAMETERS`): какие данные хотим получить;
* токен доступа (`ACCESS_TOKEN`): токен доступа;
* версия API (`V`): к какой версии API обращаемся (есть в документации методов).

Другими словами, запрос – это обычная строка, которую мы можем сформировать, вставить в адресную строку в браузере и получить результат! Однако, чтобы на наш запрос получить ответ, нужно иметь доступ к API. Иногда API бывают совсем открытыми, но чаще всего, необходимо запрашивать доступ (без доступа можно обойтись, но тогда функционал будет сильно ограничен, и мы не сможем получать содержательную информацию).

**Для получения токена доступа:**

1. Создать приложение ВКонтакте (пройти по ссылке). Дать название, выбрать *Standalone-приложение*.
2. Включить приложение, сделать публичным (*Настройки - Состояние и выбрать в выпадающем меню Приложение включено и видно всем*).
3. Авторизоваться: скопировать строку ниже в браузер, в `client id` вместо 1 выставить id своего приложения (первая строка в настроках ‒ ID приложения). Если не хочется ни в чем ограничивать свое приложение, можно оставить `scope=all` (у приложения будет доступ ко всему, к чему есть доступ у пользователя).

    `https://oauth.vk.com/authorize?client_id=1&display=page&redirect_uri=http://oauth.vk.com/blank.html&scope=all&response_type=token`

    Скопировать `access token` из обновленной адресной строки (все после `access_token=` и до `&expires_in`, без `&`). Никому не показывать! По этому токену можно получить доступ ко всему аккаунту.

Итак, теперь попробуем сохранить в переменные название метода, параметры, токен доступа и версию API.

### Пример 1: информация о пользователях

Мы попробуем получить информацию о пользователе по его имени (*screen_name*), которое мы видим в ссылке на его страницу. Для этого нам потребуется метод `users.get` (см. весь перечень методов [здесь](https://vk.com/dev/methods)), параметр `user_ids` для указания id или имени пользователя и версия 5.103 (взята из документации по методу `users.get`).

In [2]:
METHOD_NAME = "users.get"
PARAMETERS = "user_ids=allatamb"  # enter some id or screen name
ACCESS_TOKEN = "edc29623d7eb58cbedd013376922275dde535d1ec4944fc953527a17b5fdd10cb868ae0f0c7d1d9a4fe77" # enter token
V = "5.103"

Формируем запрос – подставляем в шаблон запроса (ссылка, которую мы видели в самом начале) значения переменных выше. Удобнее всего это сделать через f-строки:

In [3]:
request = f"https://api.vk.com/method/{METHOD_NAME}?{PARAMETERS}&access_token={ACCESS_TOKEN}&v={V}"

Если бы мы не пользовались Python (что неудобно, но технически возможно), мы бы скопировали ссылку выше в адресную строку браузера и посмотрели на результат:

In [4]:
print(request)

https://api.vk.com/method/users.get?user_ids=allatamb&access_token=edc29623d7eb58cbedd013376922275dde535d1ec4944fc953527a17b5fdd10cb868ae0f0c7d1d9a4fe77&v=5.103


Но мы поступим иначе – отправим запрос с помощью функции `get()` из библиотеки `requests` (как раньше подставляли в эту функцию ссылку на страницу, которую хотели парсить с помощью BeautifulSoup):

In [14]:
response = requests.get(request)
print(response)

<Response [200]>


Теперь посмотрим на содержимое ответа (*contents*):

In [15]:
response_str = response.content
print(response_str) # байтовая строка

b'{"response":[{"id":20473269,"first_name":"\xd0\x90\xd0\xbb\xd0\xbb\xd0\xb0","last_name":"\xd0\xa2\xd0\xb0\xd0\xbc\xd0\xb1\xd0\xbe\xd0\xb2\xd1\x86\xd0\xb5\xd0\xb2\xd0\xb0","is_closed":false,"can_access_closed":true}]}'


Содержимое ответа – это JSON-файл, что расшифровывается как *JavaScript Object Notation*. Изначально этот формат хранения данных использовался в языке JavaScript, но теперь он потерял привязку к конкретному языку программирования и стал универсальным. *Object* здесь можно понимать как некоторую структуру хранения данных (список, кортеж, словарь), которая записывается в специальном виде, внешне напоминающим обычную строку.

Выгрузим из `response` json-файл:

In [17]:
data = response.json()
data

{'response': [{'id': 20473269,
   'first_name': 'Алла',
   'last_name': 'Тамбовцева',
   'is_closed': False,
   'can_access_closed': True}]}

Несложно заметить, что мы получили обычный словарь. Выберем из него запись с ключом *response*, поскольку именно там хранятся все данные:

In [18]:
print(data["response"]) 

[{'id': 20473269, 'first_name': 'Алла', 'last_name': 'Тамбовцева', 'is_closed': False, 'can_access_closed': True}]


Получили список из одного элемента. Его можно извлечь:

In [19]:
print(data["response"][0]) 

{'id': 20473269, 'first_name': 'Алла', 'last_name': 'Тамбовцева', 'is_closed': False, 'can_access_closed': True}


Теперь можем вызывать из полученного словаря записи по ключам: 

In [20]:
data["response"][0]["last_name"] 

'Тамбовцева'

При желании можем преобразовать словарь в датафрейм pandas, но давайте сделаем это для более объемного примера – выберем сразу несколько пользователей и больше полей с данными:

In [21]:
users = ["allatamb", "navasilyonok"]  # список строк
users_all = ",".join(users)  # объединяем в одну строку по запятой
print(users_all) 

allatamb,navasilyonok


In [22]:
fields = ["bdate", "city", "universities"]
fields_all = ",".join(fields)
print(fields_all) 

bdate,city,universities


In [23]:
METHOD_NAME = "users.get"
PARAMETERS = "user_ids=" + users_all + '&' + 'fields=' + fields_all

In [24]:
request = f"https://api.vk.com/method/{METHOD_NAME}?{PARAMETERS}&access_token={ACCESS_TOKEN}&v={V}"

In [25]:
response = requests.get(request)
data = response.json() 
data

{'response': [{'id': 20473269,
   'first_name': 'Алла',
   'last_name': 'Тамбовцева',
   'is_closed': False,
   'can_access_closed': True,
   'bdate': '25.3.1994',
   'city': {'id': 1, 'title': 'Москва'},
   'universities': [{'id': 128,
     'country': 1,
     'city': 1,
     'name': 'НИУ ВШЭ (ГУ-ВШЭ)',
     'graduation': 2016,
     'education_status': 'Студентка (бакалавр)'}]},
  {'id': 171544496,
   'first_name': 'Наталья',
   'last_name': 'Василёнок',
   'is_closed': False,
   'can_access_closed': True,
   'bdate': '21.7.1996',
   'city': {'id': 1, 'title': 'Москва'},
   'universities': [{'id': 128,
     'country': 1,
     'city': 1,
     'name': 'НИУ ВШЭ (ГУ-ВШЭ)',
     'faculty': 475,
     'faculty_name': 'Факультет социальных наук',
     'chair': 2037672,
     'chair_name': 'Политология',
     'graduation': 2017,
     'education_form': 'Очное отделение',
     'education_status': 'Студентка (бакалавр)'}]}]}

In [26]:
df = pd.DataFrame(data['response'])
df

,id,first_name,last_name,is_closed,can_access_closed,bdate,city,universities
0,20473269,Алла,Тамбовцева,False,True,25.3.1994,"{'id': 1, 'title': 'Москва'}","[{'id': 128, 'country': 1, 'city': 1, 'name': ..."
1,171544496,Наталья,Василёнок,False,True,21.7.1996,"{'id': 1, 'title': 'Москва'}","[{'id': 128, 'country': 1, 'city': 1, 'name': ..."


In [29]:
def get_univ(x):
    try:
        y = x[0]["name"]
    except:
        y = None
    return y

In [30]:
df["City"] = df["city"].apply(lambda x: x["title"]) 
#df['Univ'] = df["universities"].apply(lambda x: x[0]["name"]) 
#df["Year"] = df["universities"].apply(lambda x: x[0]["graduation"]) 

df['Univ'] = df["universities"].apply(get_univ) 

In [31]:
df

,id,first_name,last_name,is_closed,can_access_closed,bdate,city,universities,City,Univ,Year
0,20473269,Алла,Тамбовцева,False,True,25.3.1994,"{'id': 1, 'title': 'Москва'}","[{'id': 128, 'country': 1, 'city': 1, 'name': ...",Москва,НИУ ВШЭ (ГУ-ВШЭ),2016
1,171544496,Наталья,Василёнок,False,True,21.7.1996,"{'id': 1, 'title': 'Москва'}","[{'id': 128, 'country': 1, 'city': 1, 'name': ...",Москва,НИУ ВШЭ (ГУ-ВШЭ),2017


### Пример 2:  список друзей

In [35]:
### Ваш код, маэстро
METHOD_NAME = "users.get"
PARAMETERS = "user_ids=allatamb"  # enter some id or screen name
ACCESS_TOKEN = "edc29623d7eb58cbedd013376922275dde535d1ec4944fc953527a17b5fdd10cb868ae0f0c7d1d9a4fe77" # enter token here
V = "5.103"

request = f"https://api.vk.com/method/{METHOD_NAME}?{PARAMETERS}&access_token={ACCESS_TOKEN}&v={V}"
resp1 = requests.get(request)
data1 = resp1.json()
data1['response'][0]['id']

20473269

In [ ]:
METHOD_NAME = "friends.get"
PARAMETERS = "user_ids=20473269"
request2 = f"https://api.vk.com/method/{METHOD_NAME}?{PARAMETERS}&access_token={ACCESS_TOKEN}&v={V}"
resp2 = requests.get(request2)
data2 = resp2.json()

In [42]:
friends = data2['response']['items']  
friends = [str(i) for i in friends] 

In [45]:
friends_all = ",".join(friends)
METHOD_NAME = "users.get"
PARAMETERS = "user_ids=" + friends_all
request3 = f"https://api.vk.com/method/{METHOD_NAME}?{PARAMETERS}&access_token={ACCESS_TOKEN}&v={V}"

In [46]:
resp3 = requests.get(request3)
data3 = resp3.json()
data3

{'response': [{'id': 97723,
   'first_name': 'Валерий',
   'last_name': 'Окунев',
   'is_closed': False,
   'can_access_closed': True},
  {'id': 130896,
   'first_name': 'Егор',
   'last_name': 'Юрескул',
   'is_closed': False,
   'can_access_closed': True},
  {'id': 251877,
   'first_name': 'Антон',
   'last_name': 'Воробьев',
   'is_closed': True,
   'can_access_closed': True},
  {'id': 822104,
   'first_name': 'Любовь',
   'last_name': 'Сысоева',
   'is_closed': False,
   'can_access_closed': True},
  {'id': 995866,
   'first_name': 'Антон',
   'last_name': 'Клименков',
   'is_closed': False,
   'can_access_closed': True},
  {'id': 1438878,
   'first_name': 'Dmitry',
   'last_name': 'Ekhilevskiy',
   'is_closed': False,
   'can_access_closed': True},
  {'id': 1494733,
   'first_name': 'Маша',
   'last_name': 'Пономарева',
   'is_closed': False,
   'can_access_closed': True},
  {'id': 1512070,
   'first_name': 'Виталий',
   'last_name': 'Пивиков',
   'is_closed': False,
   'can_acces

### Пример 3: посты со стены сообщества

In [47]:
group = 'hseofficial'
count = '100'

In [48]:
METHOD_NAME = "wall.get"
PARAMETERS = "domain=" + group + '&' + 'count=' + count

In [49]:
req = f"https://api.vk.com/method/{METHOD_NAME}?{PARAMETERS}&access_token={ACCESS_TOKEN}&v={V}"

In [50]:
resp = requests.get(req)
data = resp.json()

In [51]:
data

{'response': {'count': 8644,
  'items': [{'id': 33029,
    'from_id': -132,
    'owner_id': -132,
    'date': 1575971849,
    'marked_as_ads': 0,
    'post_type': 'post',
    'text': 'Что меня огорчает',
    'attachments': [{'type': 'link',
      'link': {'url': 'https://www.hse.ru/our/news/323346844.html',
       'title': 'Что меня огорчает',
       'caption': 'www.hse.ru',
       'description': 'Авторская колонка первого проректора Вадима Радаева',
       'photo': {'id': 457289885,
        'album_id': -26,
        'owner_id': 2000021743,
        'sizes': [{'type': 'k',
          'url': 'https://sun9-71.userapi.com/c858024/v858024349/120e4c/to-wJyYBh9w.jpg',
          'width': 1000,
          'height': 480},
         {'type': 'l',
          'url': 'https://sun9-32.userapi.com/c858024/v858024349/120e4b/jvKn7gctoWs.jpg',
          'width': 537,
          'height': 240},
         {'type': 'm',
          'url': 'https://sun9-12.userapi.com/c858024/v858024349/120e48/c9FrIN8g3F0.jpg',
     

In [52]:
res = data['response']['items'] 
res

[{'id': 33029,
  'from_id': -132,
  'owner_id': -132,
  'date': 1575971849,
  'marked_as_ads': 0,
  'post_type': 'post',
  'text': 'Что меня огорчает',
  'attachments': [{'type': 'link',
    'link': {'url': 'https://www.hse.ru/our/news/323346844.html',
     'title': 'Что меня огорчает',
     'caption': 'www.hse.ru',
     'description': 'Авторская колонка первого проректора Вадима Радаева',
     'photo': {'id': 457289885,
      'album_id': -26,
      'owner_id': 2000021743,
      'sizes': [{'type': 'k',
        'url': 'https://sun9-71.userapi.com/c858024/v858024349/120e4c/to-wJyYBh9w.jpg',
        'width': 1000,
        'height': 480},
       {'type': 'l',
        'url': 'https://sun9-32.userapi.com/c858024/v858024349/120e4b/jvKn7gctoWs.jpg',
        'width': 537,
        'height': 240},
       {'type': 'm',
        'url': 'https://sun9-12.userapi.com/c858024/v858024349/120e48/c9FrIN8g3F0.jpg',
        'width': 130,
        'height': 87},
       {'type': 'p',
        'url': 'https://sun

In [53]:
data['response']['items'][0] 

{'id': 33029,
 'from_id': -132,
 'owner_id': -132,
 'date': 1575971849,
 'marked_as_ads': 0,
 'post_type': 'post',
 'text': 'Что меня огорчает',
 'attachments': [{'type': 'link',
   'link': {'url': 'https://www.hse.ru/our/news/323346844.html',
    'title': 'Что меня огорчает',
    'caption': 'www.hse.ru',
    'description': 'Авторская колонка первого проректора Вадима Радаева',
    'photo': {'id': 457289885,
     'album_id': -26,
     'owner_id': 2000021743,
     'sizes': [{'type': 'k',
       'url': 'https://sun9-71.userapi.com/c858024/v858024349/120e4c/to-wJyYBh9w.jpg',
       'width': 1000,
       'height': 480},
      {'type': 'l',
       'url': 'https://sun9-32.userapi.com/c858024/v858024349/120e4b/jvKn7gctoWs.jpg',
       'width': 537,
       'height': 240},
      {'type': 'm',
       'url': 'https://sun9-12.userapi.com/c858024/v858024349/120e48/c9FrIN8g3F0.jpg',
       'width': 130,
       'height': 87},
      {'type': 'p',
       'url': 'https://sun9-5.userapi.com/c858024/v8580

In [54]:
posts = pd.DataFrame(res) 

In [55]:
posts["nlikes"] = posts["likes"].apply(lambda x: int(x["count"]))
posts["nreposts"] = posts["reposts"].apply(lambda x: int(x["count"]))
posts["nviews"] = posts["views"].apply(lambda x: int(x["count"]))

In [56]:
posts

,id,from_id,owner_id,date,marked_as_ads,post_type,text,attachments,post_source,comments,likes,reposts,views,is_favorite,copy_history,edited,signer_id,nlikes,nreposts,nviews
0,33029,-132,-132,1575971849,0,post,Что меня огорчает,"[{'type': 'link', 'link': {'url': 'https://www...",{'type': 'vk'},"{'count': 1, 'can_post': 1, 'groups_can_post':...","{'count': 12, 'user_likes': 0, 'can_like': 1, ...","{'count': 0, 'user_reposted': 0}",{'count': 6670},False,NaN,NaN,NaN,12,0,6670
1,33028,-132,-132,1575971665,0,post,,NaN,{'type': 'vk'},"{'count': 0, 'can_post': 1, 'groups_can_post':...","{'count': 14, 'user_likes': 0, 'can_like': 1, ...","{'count': 2, 'user_reposted': 0}",{'count': 3503},False,"[{'id': 400, 'owner_id': -130374393, 'from_id'...",NaN,NaN,14,2,3503
2,33027,-132,-132,1575971625,0,post,,NaN,{'type': 'vk'},"{'count': 0, 'can_post': 1, 'groups_can_post':...","{'count': 4, 'user_likes': 0, 'can_like': 1, '...","{'count': 0, 'user_reposted': 0}",{'count': 2348},False,"[{'id': 95, 'owner_id': -185974612, 'from_id':...",NaN,NaN,4,0,2348
3,33026,-132,-132,1575971586,0,post,,"[{'type': 'photo', 'photo': {'id': 457240210, ...",{'type': 'vk'},"{'count': 0, 'can_post': 1, 'groups_can_post':...","{'count': 9, 'user_likes': 0, 'can_like': 1, '...","{'count': 1, 'user_reposted': 0}",{'count': 5660},False,NaN,NaN,NaN,9,1,5660
4,33025,-132,-132,1575971433,0,post,"Вышка в прошлом году запустила новый проект, о...","[{'type': 'link', 'link': {'url': 'https://www...",{'type': 'vk'},"{'count': 0, 'can_post': 1, 'groups_can_post':...","{'count': 9, 'user_likes': 0, 'can_like': 1, '...","{'count': 0, 'user_reposted': 0}",{'count': 2292},False,NaN,NaN,NaN,9,0,2292
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,32624,-132,-132,1553244921,0,post,,NaN,{'type': 'vk'},"{'count': 1, 'can_post': 1, 'groups_can_post':...","{'count': 11, 'user_likes': 0, 'can_like': 1, ...","{'count': 3, 'user_reposted': 0}",{'count': 8160},False,"[{'id': 318, 'owner_id': -108877455, 'from_id'...",NaN,NaN,11,3,8160
96,32623,-132,-132,1553244602,0,post,В лицее НИУ ВШЭ: https://nnov.hse.ru/managemen...,NaN,{'type': 'vk'},"{'count': 0, 'can_post': 1, 'groups_can_post':...","{'count': 3, 'user_likes': 0, 'can_like': 1, '...","{'count': 1, 'user_reposted': 0}",{'count': 7777},False,"[{'id': 268, 'owner_id': -170789907, 'from_id'...",NaN,NaN,3,1,7777
97,32622,-132,-132,1553244286,0,post,,NaN,{'type': 'vk'},"{'count': 0, 'can_post': 1, 'groups_can_post':...","{'count': 3, 'user_likes': 0, 'can_like': 1, '...","{'count': 1, 'user_reposted': 0}",{'count': 6231},False,"[{'id': 324, 'owner_id': -108877455, 'from_id'...",NaN,NaN,3,1,6231
98,32621,-132,-132,1553244139,0,post,ВШЭ Нижний Новгород: https://nnov.hse.ru/manag...,"[{'type': 'link', 'link': {'url': 'https://nno...",{'type': 'vk'},"{'count': 0, 'can_post': 1, 'groups_can_post':...","{'count': 1, 'user_likes': 0, 'can_like': 1, '...","{'count': 0, 'user_reposted': 0}",{'count': 5508},False,NaN,NaN,NaN,1,0,5508


In [ ]:
# renew parameters and run

offset = '100'
PARAMETERS = "domain=" + group + '&' + 'count=' + count + 'offset=' + offset

# your code here